### Load dataset

In [24]:
%run "..\\Model\\DataHelpers.ipynb"

FILE_PATH = f"../Data/patient_genes_lasso.csv"
FILE_PATH_VALIDATION = f"../Data/validationset.csv"

GENE_FILE_VARIANT = 'lasso'
featuresLASSO = [ 'CD1A'  , 'CSF2RB', 'EPCAM' , 'ERBB2'  , 'ESR1'    ,
                  'EZH2'  , 'FGB'   , 'FOXA1' , 'FOXC1'  , 'GATA3'   , 
                  'LAMA2' , 'LMNA'  , 'MDGA2' , 'OBSCN'  , 'OGN'     , 
                  'PGR'   , 'SELL'  , 'SRC'   , 'TACSTD2', 'TBC1D22B', 
                  'TFF1'  , 'TGFB3' , 'UBE2C' , 'VTCN1'  ,'WTAP'     , 
                  'YES1'  , 'YOD1' ]

df = pd.read_csv(FILE_PATH)
dfValidation = pd.read_csv(FILE_PATH_VALIDATION)

# Validation set same balance as trainset/testset
dfValidation = pd.concat([dfValidation[dfValidation['tnbc'] ==False], dfValidation[dfValidation['tnbc'] ==True].head(15)])

total = len(ModelVariant)
counter = 1

for modelName in ModelVariant.__members__:
    print(f'{counter}/{total} - Applying {modelName} to features - Start')
    model = getModel(modelName)

    ### Dataset split: training and test data
    X, y, X_train, X_test, y_train, y_test, test_case_ids = split_data(df, "tnbc", True)
    
    y_pred, y_prod = run_model(model, X_train, X_test, y_train, y_test, test_case_ids, False, False, modelName)
    print_evaluated_model_accuracy(y_test, y_pred)

    metrics = run_cross_validation(model, X, y, y_test, y_pred, y_prod, False, False, modelName)

    ### Validation
    X_val = dfValidation[featuresLASSO]
    y_val = dfValidation['tnbc']
    test_case_ids_val = dfValidation['case_id']

    y_pred, y_prod = run_model_validation(model, X_val, y_val, test_case_ids_val)
    print_evaluated_model_accuracy(y_val, y_pred)

    metrics = run_cross_validation_validation(model, X, y, y_val, y_pred, y_prod)

    print(f'{counter}/{total} - Applying {modelName} to features - End')
    counter += 1

    

1/3 - Applying SVM to features - Start
X_train.shape=(781, 27)
X_test.shape=(196, 27)
y_train.shape=(781,)
y_test.shape=(196,)
Accuracy: 0.95
Model validation for SVC:
[0.9489795918367347, 0.9489795918367347, 0.958974358974359, 0.9487179487179487, 0.9487179487179487]

Mean accuracy: 0.9509

Accuracy: 0.91
Model validation for SVC:
[0.9489795918367347, 0.9489795918367347, 0.958974358974359, 0.9487179487179487, 0.9487179487179487]

Mean accuracy: 0.9509

1/3 - Applying SVM to features - End
2/3 - Applying RF to features - Start
X_train.shape=(781, 27)
X_test.shape=(196, 27)
y_train.shape=(781,)
y_test.shape=(196,)
Accuracy: 0.96
Model validation for RandomForestClassifier:
[0.9591836734693877, 0.9489795918367347, 0.9487179487179487, 0.9384615384615385, 0.9384615384615385]

Mean accuracy: 0.9468

Accuracy: 0.88
Model validation for RandomForestClassifier:
[0.9591836734693877, 0.9489795918367347, 0.9487179487179487, 0.9384615384615385, 0.9384615384615385]

Mean accuracy: 0.9468

2/3 - Appl

C:\Data\998_Software\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
